In [ ]:
#DuckDB connection
import duckdb

con = duckdb.connect("gdelt_h1_2019.duckdb")


In [11]:
#Laden halbes Jahr 2019 und Filtern auf Event Root Codes 14 und 18

con.execute("""
DROP TABLE IF EXISTS gdelt_events_filtered;

CREATE TABLE gdelt_events_filtered AS
SELECT
    column00 AS global_event_id,
    TRY_CAST(column01 AS INTEGER) AS sql_date,
    TRY_CAST(column03 AS INTEGER) AS year,

    TRY_CAST(column26 AS INTEGER) AS event_root_code,
    column27 AS event_base_code,
    TRY_CAST(column30 AS DOUBLE) AS goldstein_scale,

    column06 AS actor1_code,
    column16 AS actor2_code,

    column52 AS action_country
FROM read_csv(
    [
      'C:/Users/matia/Desktop/Statistik 2/Rohdaten_extrahiert/201901*.export.CSV',
      'C:/Users/matia/Desktop/Statistik 2/Rohdaten_extrahiert/201902*.export.CSV',
      'C:/Users/matia/Desktop/Statistik 2/Rohdaten_extrahiert/201903*.export.CSV',
      'C:/Users/matia/Desktop/Statistik 2/Rohdaten_extrahiert/201904*.export.CSV',
      'C:/Users/matia/Desktop/Statistik 2/Rohdaten_extrahiert/201905*.export.CSV',
      'C:/Users/matia/Desktop/Statistik 2/Rohdaten_extrahiert/201906*.export.CSV'
    ],
    delim = '\t',
    header = false,
    all_varchar = true,
    ignore_errors = true,
    null_padding = true,
    max_line_size = 10000000
)
WHERE TRY_CAST(column26 AS INTEGER) IN (14, 18);
""")



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [12]:
con.execute(r"""
SELECT *
FROM read_csv_auto(
    'C:\Users\matia\Desktop\Statistik 2\Rohdaten_extrahiert\*.CSV',
    delim='\t',
    header=False
)
LIMIT 5
""").fetchdf()


,column00,column01,column02,column03,column04,column05,column06,column07,column08,column09,...,column48,column49,column50,column51,column52,column53,column54,column55,column56,column57
0,813415774,20180101,201801,2018,2018.0027,None,None,None,None,None,...,-561990,4,"Calgary, Alberta, Canada",CA,CA01,51.0833,-114.0830,-561990,20190101,https://www.weyburnreview.com/cn-rail-and-cp-r...
1,813415775,20180101,201801,2018,2018.0027,None,None,None,None,None,...,648428,3,"Nashwauk, Minnesota, United States",US,USMN,47.3802,-93.1683,648428,20190101,http://www.startribune.com/arcelormittal-takes...
2,813415776,20180101,201801,2018,2018.0027,AGR,FARMER,None,None,None,...,None,4,"Calgary, Alberta, Canada",CA,CA01,51.0833,-114.0830,-561990,20190101,https://www.weyburnreview.com/cn-rail-and-cp-r...
3,813415777,20180101,201801,2018,2018.0027,AGR,FARMER,None,None,None,...,None,4,"Calgary, Alberta, Canada",CA,CA01,51.0833,-114.0830,-561990,20190101,https://www.weyburnreview.com/cn-rail-and-cp-r...
4,813415778,20180101,201801,2018,2018.0027,CAN,CANADA,CAN,None,None,...,None,4,"Winnipeg, Manitoba, Canada",CA,CA03,49.8833,-97.1667,-576096,20190101,https://globalnews.ca/news/4805430/professor-p...


# Spaltennamen geben

In [16]:
#Datei mit Spaltennamen einladen
import pandas as pd

mapping_path = r"C:\Users\matia\Desktop\Statistik 2\Entschlüsselung\GDELT 1.0 Event Field List (Spaltennamen).csv"

df_fields = pd.read_csv(mapping_path, sep=";")
df_fields.head()



,Field Name,Column ID,Unnamed: 2
0,GLOBALEVENTID,0,NaN
1,SQLDATE,1,NaN
2,MonthYear,2,NaN
3,Year,3,NaN
4,FractionDate,4,NaN


In [ ]:
# Mapping Dictionary erstellen
mapping = {
    f"column{int(row['Column ID']):02d}": row["Field Name"]
    for _, row in df_fields.iterrows()
}

# Kontrolle
list(mapping.items())[:10]


[('column00', 'GLOBALEVENTID'),
 ('column01', 'SQLDATE'),
 ('column02', 'MonthYear'),
 ('column03', 'Year'),
 ('column04', 'FractionDate'),
 ('column05', 'Actor1Code'),
 ('column06', 'Actor1Name'),
 ('column07', 'Actor1CountryCode'),
 ('column08', 'Actor1KnownGroupCode'),
 ('column09', 'Actor1EthnicCode')]

In [18]:
# Join mit DuckDB durchführen
select_clause = ",\n".join(
    [f"{col} AS {name}" for col, name in mapping.items()]
)

con.execute(f"""
CREATE OR REPLACE VIEW gdelt_raw_named AS
SELECT
{select_clause}
FROM read_csv_auto(
    'C:/Users/matia/Desktop/Statistik 2/Rohdaten_extrahiert/*.CSV',
    delim='\t',
    header=False
)
""")


In [ ]:
#Test ob funktioniert hat
con.execute("""
SELECT *
FROM gdelt_raw_named
LIMIT 5
""").fetchdf()


,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,813415774,20180101,201801,2018,2018.0027,None,None,None,None,None,...,-561990,4,"Calgary, Alberta, Canada",CA,CA01,51.0833,-114.0830,-561990,20190101,https://www.weyburnreview.com/cn-rail-and-cp-r...
1,813415775,20180101,201801,2018,2018.0027,None,None,None,None,None,...,648428,3,"Nashwauk, Minnesota, United States",US,USMN,47.3802,-93.1683,648428,20190101,http://www.startribune.com/arcelormittal-takes...
2,813415776,20180101,201801,2018,2018.0027,AGR,FARMER,None,None,None,...,None,4,"Calgary, Alberta, Canada",CA,CA01,51.0833,-114.0830,-561990,20190101,https://www.weyburnreview.com/cn-rail-and-cp-r...
3,813415777,20180101,201801,2018,2018.0027,AGR,FARMER,None,None,None,...,None,4,"Calgary, Alberta, Canada",CA,CA01,51.0833,-114.0830,-561990,20190101,https://www.weyburnreview.com/cn-rail-and-cp-r...
4,813415778,20180101,201801,2018,2018.0027,CAN,CANADA,CAN,None,None,...,None,4,"Winnipeg, Manitoba, Canada",CA,CA03,49.8833,-97.1667,-576096,20190101,https://globalnews.ca/news/4805430/professor-p...
